In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# 1. Load and preprocess data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 2. Define the improved CNN with Batch Normalization
def improved_cnn_with_bn():
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        # Block 2
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.4),

        # Dense Layers
        layers.Flatten(),
        layers.Dense(512),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 3. Compile and train the model
model = improved_cnn_with_bn()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test), verbose=1)

# 4. Evaluate accuracy
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f" Test Accuracy: {accuracy:.4f}")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step - accuracy: 0.3789 - loss: 1.8305 - val_accuracy: 0.3465 - val_loss: 2.3236
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.6150 - loss: 1.0662 - val_accuracy: 0.6229 - val_loss: 1.0851
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6892 - loss: 0.8788 - val_accuracy: 0.5744 - val_loss: 1.3161
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7298 - loss: 0.7716 - val_accuracy: 0.7276 - val_loss: 0.8019
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7587 - loss: 0.6926 - val_accuracy: 0.7511 - val_loss: 0.7294
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7765 - loss: 0.6367 - val_accuracy: 0.7322 - val_loss: 0.7866
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7932 - loss: 0.5928 - val_accuracy: 0.7950 - val_loss: 0.5965
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.8008 - loss: 0.5610 - val_accu

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Reduce dataset for speed (optional)
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:1000], y_test[:1000]

# Preprocess and resize
x_train = tf.image.resize(x_train, [224, 224])
x_test = tf.image.resize(x_test, [224, 224])
x_train = preprocess_input(x_train)

x_test = preprocess_input(x_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [ ]:
# Load base ResNet50 (frozen)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile & train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f" Test Accuracy: {test_acc:.4f}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 184ms/step - accuracy: 0.5783 - loss: 1.2687 - val_accuracy: 0.8600 - val_loss: 0.4549
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 25s 103ms/step - accuracy: 0.8870 - loss: 0.3502 - val_accuracy: 0.8750 - val_loss: 0.3792
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 126ms/step - accuracy: 0.9245 - loss: 0.2412 - val_accuracy: 0.8670 - val_loss: 0.3859
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 109ms/step - accuracy: 0.9422 - loss: 0.1908 - val_accuracy: 0.8710 - val_loss: 0.3778
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 130ms/step - accuracy: 0.9599 - loss: 0.1504 - val_accuracy: 0.8810 - val_loss: 0.3646
 Test Accuracy: 0.8750


#Comparison: Custom CNN vs ResNet50 on CIFAR-10
| **Aspect**              | **Custom CNN (from scratch)**                       | **ResNet50 (transfer learning)**                    |
| ----------------------- | --------------------------------------------------- | --------------------------------------------------- |
| **Architecture**        | Manually designed with Conv layers, BatchNorm, etc. | Pre-trained ResNet50 base + custom classifier head  |
| **Input Image Size**    | 32×32                                               | Resized to 224×224                                  |
| **Training Time**       | Relatively fast (smaller model)                     | Slower (due to larger model & resizing overhead)    |
| **Accuracy (Test Set)** | 85.16%                                           | 87.50%            |
| **Resource Usage**      | Lightweight – good for limited GPU/CPU              | Heavy – requires more memory and compute            |
| **Flexibility**         | Fully controllable and modifiable                   | Less flexible (pre-defined layers, limited edits)   |
| **Best For**            | Learning, experimentation, lightweight deployment   | Higher accuracy on small datasets, production-ready |

Summary:

The Custom CNN is faster and resource-efficient, making it great for learning, prototyping, and lower-end hardware.

ResNet50 provides better accuracy due to its deep pre-trained architecture, but at the cost of higher computational demands and training time.